# Connection Workflow (Sanitized)
This notebook is safe for sharing: it uses environment variables and does not print secrets.

In [ ]:
import os
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
client_secret_file = os.getenv("GOOGLE_CLIENT_SECRET_FILE", "clientsecret.json")
sheet_name = os.getenv("GOOGLE_SHEET_NAME", "YOUR_GOOGLE_SHEET_NAME")

creds = ServiceAccountCredentials.from_json_keyfile_name(client_secret_file, scope)
client = gspread.authorize(creds)
sheet = client.open(sheet_name).sheet1
df = pd.DataFrame(sheet.get_all_records())
df.head()

In [ ]:
import os
import mysql.connector

conn = mysql.connector.connect(
    host=os.getenv("DB_HOST", "localhost"),
    user=os.getenv("DB_USER", "root"),
    password=os.getenv("DB_PASSWORD", ""),
    database=os.getenv("DB_NAME", "survey_db"),
    use_pure=True
)

cursor = conn.cursor()
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO survey_responses (
            timestamp, play_frequency, platform, genres, matters_most, preference,
            story_importance, romance_importance, romance_engagement, romance_preference,
            player_gender, identity, orientation_importance, orientation, inclusive_interest
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, tuple(row))
conn.commit()
cursor.close()
conn.close()